In [0]:
CREATE OR REFRESH MATERIALIZED VIEW products.sales_gold.total_sales_by_customer_${env}
TBLPROPERTIES ("quality" = "gold")
COMMENT "Ventas totales y cantidad de órdenes por cliente"
AS
SELECT 
  c.customer_id,
  c.name AS customer_name,
  COUNT(DISTINCT o.order_id) AS total_orders,
  SUM(i.total_price) AS total_spent
FROM products.sales_silver.orders_clean_${env} o
JOIN products.sales_bronze.customers_dim_${env} c ON o.customer_id = c.customer_id
JOIN products.sales_silver.order_items_clean_${env} i ON o.order_id = i.order_id
GROUP BY c.customer_id, c.name;

CREATE OR REFRESH MATERIALIZED VIEW products.sales_gold.total_sales_by_product_${env}
TBLPROPERTIES ("quality" = "gold")
COMMENT "Ventas totales y cantidad vendida por producto"
AS
SELECT 
  p.product_id,
  p.name AS product_name,
  COUNT(i.order_id) AS total_sales,
  SUM(i.quantity) AS total_quantity,
  SUM(i.total_price) AS total_revenue
FROM products.sales_silver.order_items_clean_${env} i
JOIN products.sales_bronze.products_dim_${env} p ON i.product_id = p.product_id
GROUP BY p.product_id, p.name;

CREATE OR REPLACE VIEW products.sales_gold.orders_view_${env}
TBLPROPERTIES ("quality" = "gold")
COMMENT "Vista de los orders"
AS
SELECT 
  order_id
FROM products.sales_silver.order_items_clean_${env};